In [1]:
import sys
print(sys.path)

['/Users/yannickaaron/Github/NovaProgrammingProject/notebooks', '/Users/yannickaaron/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles', '/Users/yannickaaron/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles/lib/python', '/Users/yannickaaron/.pyenv/versions/3.9.13/lib/python39.zip', '/Users/yannickaaron/.pyenv/versions/3.9.13/lib/python3.9', '/Users/yannickaaron/.pyenv/versions/3.9.13/lib/python3.9/lib-dynload', '', '/Users/yannickaaron/.pyenv/versions/3.9.13/lib/python3.9/site-packages']


In [2]:
#add ../src to the path
sys.path.append('../src')

In [3]:
from scraper import WCHistoricScoresScraper

In [4]:
myscraper = WCHistoricScoresScraper('fevent', 'fhome', 'fscore', 'faway')

In [5]:
years = [2018]

In [6]:
df_raw = myscraper.getAllData(years)
df_raw

,team1,team2,year,score_team1,score_team2
0,Russia,Saudi Arabia,2018,5,0
1,Egypt,Uruguay,2018,0,1
2,Russia,Egypt,2018,3,1
3,Uruguay,Saudi Arabia,2018,1,0
4,Uruguay,Russia,2018,3,0
...,...,...,...,...,...
59,Russia,Croatia,2018,2,2
60,France,Belgium,2018,1,0
61,Croatia,England,2018,2,1
62,Belgium,England,2018,2,0


In [7]:
from scraper import CurrentWCScrapper

In [8]:
currentwc = CurrentWCScrapper()

In [9]:
currentwc.createDictTable(2022)['A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Netherlands,3,2,1,0,5,1,+4,7,Advanced to knockout stage
1,2,Senegal,3,2,0,1,5,4,+1,6,Advanced to knockout stage
2,3,Ecuador,3,1,1,1,4,3,+1,4,NaN
3,4,Qatar (H),3,0,0,3,1,7,−6,0,NaN


# Analysis

In [10]:
df = df_raw.copy()

In [11]:
df['all_goals'] = df['score_team1'] + df['score_team2']

In [12]:
df_team1 = df[['team1', 'score_team1', 'score_team2','all_goals']].rename(columns={'team1':'team', 'score_team1':'score', 'score_team2':'opponent_score'})
df_team2 = df[['team2', 'score_team2','score_team1' ,'all_goals']].rename(columns={'team2':'team', 'score_team2':'score', 'score_team1':'opponent_score'})

In [13]:
import pandas as pd

In [14]:
#concatenate the two dataframes
df_all = pd.concat([df_team1, df_team2])
df_all

,team,score,opponent_score,all_goals
0,Russia,5,0,5
1,Egypt,0,1,1
2,Russia,3,1,4
3,Uruguay,1,0,1
4,Uruguay,3,0,3
...,...,...,...,...
59,Croatia,2,2,4
60,Belgium,0,1,1
61,England,1,2,3
62,England,0,2,2


In [15]:
#remove all spaces at the start or the end of a string
df_all['team'] = df_all['team'].str.strip()

In [16]:
df_all[df_all['team'] == "France"].shape[0]

7

In [17]:
df_all['team']

0      Russia
1       Egypt
2      Russia
3     Uruguay
4     Uruguay
       ...   
59    Croatia
60    Belgium
61    England
62    England
63    Croatia
Name: team, Length: 128, dtype: object

In [18]:
df_all = df_all.groupby('team').mean()

In [19]:
df_all

,score,opponent_score,all_goals
team,,,
Argentina,1.500000,2.250000,3.750000
Australia,0.666667,1.666667,2.333333
Belgium,2.285714,0.857143,3.142857
Brazil,1.600000,0.600000,2.200000
Colombia,1.500000,0.750000,2.250000
Costa Rica,0.666667,1.666667,2.333333
Croatia,2.000000,1.285714,3.285714
Denmark,0.750000,0.500000,1.250000
Egypt,0.666667,2.000000,2.666667


In [20]:
from scipy.stats import poisson
def predict_points(home, away):
    if home in df_all.index and away in df_all.index:
        # goals_scored * goals_conceded
        lamb_home = df_all.at[home,'score'] * df_all.at[away,'opponent_score']
        lamb_away = df_all.at[away,'score'] * df_all.at[home,'opponent_score']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [21]:
from analyser import WCAnalyser

In [22]:
df

,team1,team2,year,score_team1,score_team2,all_goals
0,Russia,Saudi Arabia,2018,5,0,5
1,Egypt,Uruguay,2018,0,1,1
2,Russia,Egypt,2018,3,1,4
3,Uruguay,Saudi Arabia,2018,1,0,1
4,Uruguay,Russia,2018,3,0,3
...,...,...,...,...,...,...
59,Russia,Croatia,2018,2,2,4
60,France,Belgium,2018,1,0,1
61,Croatia,England,2018,2,1,3
62,Belgium,England,2018,2,0,2


In [23]:
analyser = WCAnalyser(df_raw)

In [24]:
analyser.print_team_statistics('France')



Statistics for France:
Total Games Played: 7
Total Goals Scored: 14
Total Goals Conceded: 6
Total Wins: 6
Total Losses: 0
Total Draws: 1
Win/Loss Ratio: 6
Average Goals Scored: 2.0
Average Goals Conceded: 0.8571428571428571
Average Goals per Game: 2.857142857142857
Average Goals Difference: 1.1428571428571428
Average Score: 2.0
Number of World Cups: 1


In [25]:
analyser.predict_outcome('France', 'Germany')

Probability of France winning: 0.8170246349240524
Probability of Germany winning: 0.056811618352798104


(2.577129750056215, 0.29649070034245206)